# Richards equation

Let $\Omega=(0,2)\times(0,3)$ with boundary $\partial \Omega$ and outward unit normal ${\nu}$. Given 
$k$ the matrix permeability, we want to solve the following problem: find $({q}, \psi)$ such that
$$
\left\{
\begin{array}{ll}
\begin{array}{l} 
K^{-1}(\psi) {q} + \nabla \psi = -\nabla z\\
\partial_t \theta (\psi) + \nabla \cdot {q} = 0
\end{array}
&\text{in } \Omega \times (0,T)
\end{array}
\right.
$$
with boundary conditions:

$$
\psi(t,x,z)=
\left\{
\begin{array}{ll}
\begin{array}{l} 
-2+2.2 \frac{t}{\Delta t_D}, \text{ on } \Gamma_{D_1}, t\leq\Delta t_D\\
0.2, \text{ on } \Gamma_{D_1}, t>\Delta t_D\\
1-z, \text{ on } \Gamma_{D_2}
\end{array}
\end{array}
\right., \qquad \nu \cdot q = 0 \text{ on } \Gamma_N \qquad \psi(0,x,z) = 1-z \text{ on } \Omega$$
and
$$
\Gamma_{D_1} = \left\{ (x,z) \in \partial \Omega \:|\: x \in [0,1] \wedge z=3  \right\},\\
\Gamma_{D_2} = \left\{ (x,z) \in \partial \Omega \:|\: x = 2 \wedge z \in [0,1]  \right\},\\
\Gamma_{D} = \Gamma_{D_1} \cup \Gamma_{D_2},\\
\Gamma_{N} = \partial \Omega \setminus \Gamma_D
$$

In [1]:
import shutil
import os

import numpy as np

import porepy as pp
import pygeon as pg

import time

/opt/conda/lib/python3.11/site-packages/porepy/numerics/nonlinear/nonlinear_solvers.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import trange  # type: ignore


In [2]:
from richards.model_params import Model_Data
from richards.matrix_computer import Matrix_Computer

from richards.solver import Solver
from richards.solver_params import Solver_Data, Solver_Enum, Norm_Error

In [3]:
# Set the maximum number of iterations of the non-linear solver
K = 500

# L-scheme parameter
L = 3.501e-2

# Set the mesh refinment
N = 40

# Set the number of steps (excluding the initial condition)
num_steps = 9

# Simulation time length
T = num_steps/48

# Time switch conditions (for the boundary condition)
dt_D = 3/48

# Fluid density
rho = 1000

# Relative and absolute tolerances for the non-linear solver
abs_tol = 1e-5
rel_tol = 1e-5

# Domain tolerance
domain_tolerance = 1 / (10 * N)

# Output directory
output_directory = 'dual_single_stage'

In [4]:
# Van Genuchten model parameters ( relative permeability model )
model_data = Model_Data(theta_r=0.131, theta_s=0.396, alpha=0.423, n=2.06, K_s=4.96e-2, T=T, num_steps=num_steps)

In [5]:
# Prepare the domain and its mesh
subdomain = pp.StructuredTriangleGrid([2*N, 3*N], [2,3])

# Convert it to a mixed-dimensional grid
mdg = pp.meshing.subdomains_to_mdg([subdomain])

In [6]:
key = "flow"

# Collection of boundary conditions
bc_value = []
bc_essential = []

# Initial pressure
initial_pressure = []

# Discretizations for q and \psi
RT0 = pg.RT0(key)
P0  = pg.PwConstants(key)

In [7]:
# Initial pressure function
def initial_pressure_func(x): 
    return 1#-x[1]

In [8]:
subdomain, data = mdg.subdomains(return_data=True)[0]

# Prepare the inital pressure term by interpolating initial_pressure_func into the P0 space
initial_pressure.append(P0.interpolate(subdomain, initial_pressure_func))
        
# Get the boundary faces ids
boundary_faces_indexes = subdomain.get_boundary_faces()

# Gamma_D1 and Gamma_D2 boundary faces
gamma_d1 = np.logical_and(subdomain.face_centers[0, :] > 0-domain_tolerance, np.logical_and(subdomain.face_centers[0, :] < 1+domain_tolerance, subdomain.face_centers[1, :] > 3-domain_tolerance))
gamma_d2 = np.logical_and(subdomain.face_centers[0, :] > 2-domain_tolerance, np.logical_and(subdomain.face_centers[1, :] > 0-domain_tolerance, subdomain.face_centers[1, :] < 1+domain_tolerance))

gamma_d  = np.logical_or(gamma_d1, gamma_d2)

# Gamma_N is the remaining part of the boundary    
gamma_n  = gamma_d.copy()
gamma_n[boundary_faces_indexes] = np.logical_not(gamma_n[boundary_faces_indexes])
    
# Set the initial conductivity tensor in data (the actual saved tensor does not matter at this stage)
pp.initialize_data(subdomain, data, key, {
    "second_order_tensor": pp.SecondOrderTensor(np.ones(subdomain.num_cells)),
})
    
# Prepare the \hat{\psi} function
def bc_gamma_d(x, t):
    if   x[0] > 2-domain_tolerance and x[1] > 0-domain_tolerance and x[1] < 1+domain_tolerance:
        res =  1#-x[1]
    elif x[1] > 3-domain_tolerance and x[0] > 0-domain_tolerance and x[0] < 1+domain_tolerance:
        res = min( 3.2, 1 + 2.2 * t / dt_D )#-3
    else:
        res = 0
        
    return res

# Add a lambda function that generates for each time instant the (discretized) natural boundary conditions for the problem
bc_value = lambda t: - RT0.assemble_nat_bc(subdomain, lambda x: bc_gamma_d(x,t), gamma_d)

# Set the essential boundary conditions (they will be enforced before solving the system)
essential_pressure_dofs = np.zeros(P0.ndof(subdomain), dtype=bool)
bc_essential = np.hstack((gamma_n, essential_pressure_dofs))

In [9]:
if os.path.exists(output_directory):
    shutil.rmtree(output_directory)

In [10]:
# assemble initial solution
cp = Matrix_Computer(mdg)
initial_solution = np.zeros(cp.dof_RT0 + cp.dof_P0)
initial_solution[-cp.dof_P0:] += np.hstack(initial_pressure)

In [11]:
solver_data = Solver_Data(mdg=mdg, 
                          initial_solution=initial_solution, 
                          scheme=Solver_Enum.PICARD, L_Scheme_value=L, 
                          bc_essential=lambda t: bc_essential, 
                          eps_psi_abs=abs_tol, eps_psi_rel=rel_tol, 
                          max_iterations_per_step=K,   
                          output_directory=output_directory,
                          step_output_allowed=True,
                          report_name='dual', 
                          norm_error=Norm_Error.L2)

solver_data.set_rhs_vector_q(bc_value)
#solver_data.set_rhs_function_q(lambda x,t: np.array([0, -1, 0]))

In [12]:
solver = Solver(model_data=model_data, solver_data=solver_data)

In [13]:
start = time.time()
solver.solve()
end = time.time()

print('')
print(end - start)

Csv_Exporter: A file with name dual_PICARD_richards_solver.csv is detected. I'll delete it
Time 0.02083
Iteration #0001, relative norm of the error:    0.046901983, norm of the error:    0.114885927
Iteration #0002, relative norm of the error:    0.011076894, norm of the error:    0.027336758
Iteration #0003, relative norm of the error:    0.001836274, norm of the error:    0.004542712
Iteration #0004, relative norm of the error:    0.000180301, norm of the error:    0.000446227
Iteration #0005, relative norm of the error:    0.000012795, norm of the error:    0.000031667

Time 0.04167
Iteration #0001, relative norm of the error:    0.072475198, norm of the error:    0.179374794
Iteration #0002, relative norm of the error:    0.025381916, norm of the error:    0.064033077
Iteration #0003, relative norm of the error:    0.006766900, norm of the error:    0.017205199
Iteration #0004, relative norm of the error:    0.001143278, norm of the error:    0.002912391
Iteration #0005, relative n